In [1]:
!wget -q https://github.com/CISC-372/Notebook/releases/download/a4/test.csv
!wget -q https://github.com/CISC-372/Notebook/releases/download/a4/train.csv

In [2]:
# comment your understanding of each function 
import pandas as pd
import csv


xy_train_df = pd.read_csv('train.csv')
x_test_df  = pd.read_csv('test.csv', index_col='id')


xy_train_df['length'] = xy_train_df.apply(lambda x: len(x.review), axis=1)
xy_train_df = xy_train_df.sort_values('length')
xy_train_df

,id,rating,review,length
6037,2596,1,Five Stars_GOOD,15
5353,4643,1,Love it_Love it,15
2545,8791,1,Five Stars_Good,15
3902,6098,1,Five Stars_love!,16
2850,4609,1,love these_so cute!,19
...,...,...,...,...
5651,518,1,"So far, it's awesome_Ok, so I'll say up front ...",5765
1615,124,1,It Works (Read Tips For Potential Effectivenes...,6740
5046,7257,1,An exquisitely effective product with an astou...,8082
4859,7555,1,Gorgeous professional looking manicure at home...,8134


In [3]:
# comment your understanding of each function and each parameter below:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

vocab_size = 10000
max_len = 256

xy_train, xy_validation = train_test_split(
    xy_train_df, test_size=0.2)

# build vocabulary from training set
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(xy_train.review)


def _preprocess(texts):
  return pad_sequences(
      tokenizer.texts_to_sequences(texts),
      maxlen=max_len, 
      padding='post'
  )


x_train = _preprocess(xy_train.review)
y_train = xy_train.rating

x_valid = _preprocess(xy_validation.review)
y_valid = xy_validation.rating

x_test = _preprocess(x_test_df.review)

print(x_train.shape)
print(x_valid.shape)
print(x_test.shape)

(4978, 256)
(1245, 256)
(2667, 256)


In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
import collections
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras


import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# comment your understanding of each line and 
# the output shape of each line below. for each dimensionality, explains its 
# meaning. (e.g. None is the batch size)

x = keras.Input((max_len))

embeded = keras.layers.Embedding(vocab_size, 20)(x)

averaged = tf.reduce_mean(embeded, axis=1)

pred = keras.layers.Dense(1, activation=tf.nn.sigmoid)(averaged)

model = keras.Model(x, pred)

model.compile(
    optimizer=Adam(clipnorm=4.),
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [5]:
history = model.fit(x_train,
                    y_train,
                    epochs=5,
                    batch_size=64,
                    validation_data=(x_valid, y_valid),
                    verbose=1)

Epoch 1/5
78/78 [==============================] - 2s 10ms/step - loss: 0.6611 - accuracy: 0.7026 - val_loss: 0.5413 - val_accuracy: 0.8691
Epoch 2/5
78/78 [==============================] - 0s 6ms/step - loss: 0.5025 - accuracy: 0.8742 - val_loss: 0.4223 - val_accuracy: 0.8691
Epoch 3/5
78/78 [==============================] - 0s 6ms/step - loss: 0.3925 - accuracy: 0.8821 - val_loss: 0.3885 - val_accuracy: 0.8691
Epoch 4/5
78/78 [==============================] - 0s 6ms/step - loss: 0.3847 - accuracy: 0.8693 - val_loss: 0.3837 - val_accuracy: 0.8691
Epoch 5/5
78/78 [==============================] - 0s 6ms/step - loss: 0.3564 - accuracy: 0.8825 - val_loss: 0.3820 - val_accuracy: 0.8691


In [6]:
model.evaluate(x_valid, y_valid)

39/39 [==============================] - 0s 2ms/step - loss: 0.3820 - accuracy: 0.8691


[0.3819705843925476, 0.8690763115882874]

In [10]:
def predict_class(_dataset):
  classes = model.predict(_dataset) > 0.5
  return np.squeeze(classes * 1) 

y_predict = predict_class(x_valid)

from sklearn.metrics import  f1_score
from sklearn.metrics import confusion_matrix

print(f1_score(y_valid, y_predict, average='micro'))

0.8690763052208834


In [12]:
# submission
pd.DataFrame(
    {'id': x_test_df.index,
     'rating': predict_class(x_test)}).to_csv('sample_submission.csv', index=False)